In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql.types import *
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as f
import time

import pandas as pd
import numpy as np

In [2]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .master('local[*]') \
#     .config("spark.driver.memory", "15g") \
#     .appName('my-cool-app') \
#     .getOrCreate()

In [3]:
sc = SparkContext('local')
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [4]:
csv_list = ["09","10","11","12"]
df_list = []
activityList = [1001,1003,1004,1007,1008,1010,1099,1101,1102,1104,1105,1199,1201,1202,1203,1204,1205,1206,1207,1208,1299,2002,2003,2004,2099,2104,2107,2109,2110,2111,2112,2113,2199,2401,2402,2403,2404,2406,2407,2499,2299,2312,2317,4113,7104,7299,4001,6014,6019,6020,6018,6113,7101,7102,7103,7105,7106]

for listname in csv_list:
    df = sqlContext.read.format('com.databricks.spark.csv')\
        .options(header='true', inferSchema='true')\
        .load('C:/ssafy/2nd/s03p22a305/bigdata/analysis/2019-'+listname+'-UTF.csv')\
        .cache()
    
    # 기업 데이터 제거
    df = df.filter(df.개인기업구분 != "기업")
    df = df.filter(df.성별 != "0.기타")
    
    # 활동 분류
    df = df.filter(df.가맹점업종코드.isin(activityList))
    
    removeColList = ["회원_시도명", "회원_시군구명", "개인기업구분", "매출년월일", "가맹점_시도명", "가맹점_시군구명", "가맹점_읍면동명","결제금액", "결제건수"]
    for ls in removeColList:
        df = df.drop(ls)
    df.persist()
    df_list.append(df)

In [5]:
start = time.time()  # 시작 시간 저장

df_list[0].show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

+------+------+-----------+--------------+------+
|  성별|  연령|승인시간대1|가맹점업종코드|회원수|
+------+------+-----------+--------------+------+
|1.남성|4.40대|         13|          2004|     3|
|1.남성|4.40대|          6|          2104|     3|
|1.남성|4.40대|          7|          2104|     5|
|1.남성|4.40대|         11|          2104|     3|
|1.남성|4.40대|          6|          2104|     5|
|1.남성|4.40대|         17|          2104|     3|
|1.남성|4.40대|          7|          2104|     6|
|1.남성|4.40대|         11|          2104|     3|
|1.남성|4.40대|          7|          2104|     5|
|1.남성|4.40대|          6|          2104|     5|
|1.남성|4.40대|          7|          2104|     6|
|1.남성|4.40대|          7|          2104|     4|
|1.남성|4.40대|          6|          2104|     6|
|1.남성|5.50대|         11|          2104|     3|
|1.남성|4.40대|          7|          2104|     5|
|1.남성|4.40대|          6|          2104|     3|
|1.남성|4.40대|         11|          2104|     3|
|1.남성|2.20대|         11|          2104|     3|
|1.남성|4.40대|          6

In [6]:
# schema = StructType([StructField("성별", StringType(), True), StructField("연령", StringType(), False),\
#                      StructField("승인시간대1", IntegerType(), True), StructField("가맹점업종코드", IntegerType(), True)])

# df_all = sqlContext.createDataFrame([],schema)
df_all = df_list[0]
# df_all.show()

In [7]:
for idx, df in enumerate(df_list):
    if(idx == 0):
        continue
    print(idx)
    df_all = df_all.union(df)

1
2
3


df_all.show()

In [8]:
df_all.colRegex

<bound method DataFrame.colRegex of DataFrame[성별: string, 연령: string, 승인시간대1: int, 가맹점업종코드: int, 회원수: int]>

In [9]:
df_all_renamed = df_all.withColumnRenamed("성별", "gender")\
                        .withColumnRenamed("연령", "age")\
                        .withColumnRenamed("승인시간대1", "time")\
                        .withColumnRenamed("가맹점업종코드", "code")\
                        .withColumnRenamed("회원수", "n")                        

df_all_renamed.show()

In [10]:
rdd = df_all_renamed.rdd
rdd

MapPartitionsRDD[87] at javaToPython at NativeMethodAccessorImpl.java:0

rdd.take(3)

In [11]:
rdd = rdd.map(lambda x: Row(gender=int(x[0][0]),\
                      age=int(x[1][0]),\
                      time = int(x[2]),\
                      code =int(x[3]),\
                        n = int(x[4])))

In [12]:
rdd.take(3)[0]

Row(gender=1, age=4, time=13, code=2004, n=3)

type(rdd)

In [13]:
schema = StructType([StructField("gender", StringType(), True), StructField("age", StringType(), True),\
                     StructField("time", IntegerType(), True), StructField("code", IntegerType(), True), StructField("n", IntegerType(), True)])


In [14]:
# df_mapped = spark.createDataFrame(rdd, schema).cache()
df_mapped = spark.createDataFrame(rdd, schema).persist()

# 캐싱 없이 쇼
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

# 캐싱 없이 재차 쇼
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

# 캐싱 후 쇼
df_mapped.persist()
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

df_mapped.unpersist()

start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [15]:
df_mapped.registerTempTable("df_tmp")

df_mapped.count()

In [16]:
# 코드 형식
# df_mapped.groupBy('gender', 'age','time', 'code').count().select('gender','age','time', f.col('count').alias('n')).show()

# df_mapped.groupBy('gender', 'age','time', 'code').sum('n').show() # count().select('gender','age','time', f.col('count').alias('n')).show()
df_group = df_mapped.groupBy('gender', 'age','time', 'code').sum('n')

# 쿼리 형식
df_group = sqlContext.sql("select *, sum(n) as cnt from df_tmp group by gender, age, time, code")

In [17]:
df_group = df_group.withColumnRenamed("sum(n)", "total").persist()
df_group.columns

['gender', 'age', 'time', 'code', 'total']

In [18]:
df_group.show()

+------+---+----+----+------+
|gender|age|time|code| total|
+------+---+----+----+------+
|     1|  3|  17|2004|125796|
|     1|  2|  20|2099| 91913|
|     1|  3|  23|2199| 56757|
|     1|  1|  17|6018|169082|
|     1|  2|  11|2109|  9538|
|     1|  5|  12|2107|  4914|
|     2|  5|  16|2499| 10511|
|     1|  2|   3|2002|   565|
|     1|  6|  17|2002|  1899|
|     1|  2|   8|2499|   629|
|     2|  6|  18|2002|  1645|
|     1|  5|   9|2499|   312|
|     2|  3|   2|4113|    35|
|     2|  1|  20|6018|  4548|
|     2|  2|   1|6018|  6586|
|     1|  2|  19|7106|  8827|
|     1|  2|  17|7106|  6203|
|     2|  1|  12|6018|  1013|
|     2|  5|  18|7105|  1169|
|     1|  2|   6|2099|  1244|
+------+---+----+----+------+
only showing top 20 rows



In [19]:
df_group.registerTempTable("df_group")

In [20]:
gender = 1
age = 2
time = 17


In [21]:
query = "select code, total from df_group where gender="+str(gender)+" and age="+str(age)+" and time="+str(time) +" order by total desc"
query

'select code, total from df_group where gender=1 and age=2 and time=17 order by total desc'

In [22]:
selected_df = sqlContext.sql(query).persist()

# selected_df.show()

selected_df.persist()

In [23]:
selected_df.columns

['code', 'total']

selected_df.describe()

In [24]:
selected_df.show()

+----+------+
|code| total|
+----+------+
|6018|425968|
|2004|301919|
|2199|187756|
|2104|144818|
|2099| 95152|
|2002| 74785|
|2003| 38388|
|7102| 34156|
|1004| 17930|
|6014| 13820|
|2109| 10932|
|2107|  9322|
|2110|  7526|
|6113|  7371|
|7106|  6203|
|4001|  5370|
|4113|  3679|
|2112|  2610|
|2406|  1150|
|2499|  1097|
+----+------+
only showing top 20 rows



In [41]:
total_sum = 0
for row in selected_df.collect():
    total_sum += row.total
total_sum

1393373

In [107]:
code_df = sqlContext.read.format('com.databricks.spark.csv')\
        .options(header='true', inferSchema='true')\
        .load('C:/ssafy/2nd/s03p22a305/bigdata/analysis/codelist-UTF.csv')\
        .cache()

In [118]:
code_df.show()

+--------+-----------+------------------+----+----------------------------------+
|업종분류|       업종|                업|코드|                  적용범위 및 기준|
+--------+-----------+------------------+----+----------------------------------+
|  의생활|       의류|          맞춤복점|1001|남성복 맞춤업소,여성복 맞춤업소...|
|  의생활|       의류|            한복점|1003|  남.여 한복 맞춤업소, 개량한복...|
|  의생활|       의류|          기성복점|1004|  의류대리점(모피 및 교복 포함)...|
|  의생활|       의류|  아동 및 유아복점|1007|           아동복 및 유아복 판매점|
|  의생활|       의류|        내의판매점|1008|    내의,타월,양말,스타킹등 판매점|
|  의생활|       의류|            양품점|1010|             각종 양품,잡화 판매점|
|  의생활|       의류|         기타 의류|1099|                              null|
|  의생활|직물/침구류|        옷감판매점|1101|  양복,양장,한복등의 옷감 및 면...|
|  의생활|직물/침구류|침구,커튼,카펫트점|1102|  이불,담요,베개,카바,커튼,카펫...|
|  의생활|직물/침구류|            수예점|1104|   각종 수예품(자수,실포함) 판매점|
|  의생활|직물/침구류|            자석요|1105|              자석요 수입,제조판매|
|  의생활|직물/침구류|         기타 직물|1199|                              null|
|  의생활|   신변잡화|        악세사리점|1